$$Гайд$$
    $$(1)$$ Для создания отчета берется текущий месяц и 28 дней включительно следующего месяца.
    $$(2)$$ При формировании отчета указываются даты для отсчета указывается первое число текущего месяца, далее указывается первая дата следующего месяца (для анализа текущего месяца).
    $$(3)$$ Не допускается использовать спец символ "/", только в другую сторону. А указанный символ "/", использовать можно.


$$Доработки$$
v1.7 $$(1)$$ изменен блок по загрузке файлов, теперь необходимо указать в какой папке находятся файлы для загрузки. Если в данной папке есть файлы других форматов, то можно указать расширение искомых файлов. Если в папке нет лишних файлов и папок, то можно поставить символ ".". Поиск работает, как регулярка.
     $$(2)$$ изменен ввод даты, теперь оно вводятся при запуске программы, а не правятся в самой программе.
     $$(3)$$ Блок с вопросами и пред обработки для загрузки будущих баз перемещены вверх кода.
     $$(4)$$ Выгрузка в файл unikal_terminal перечень Уникальных терминалов.
     $$(5)$$ В блоке Стоп-листа название столбца измененно с "нам нужны 1 и более" на "нам нужны 2 и более"

    Загружаем библиотеки с формулами/функциями для последующих вычислений

In [1]:
import numpy as np #для работы с вычислениями
import pandas as pd #для работы с матрицами
import re #для поиска нужных значений в ячейках матрицы
from datetime import datetime, timedelta #для работы с датыми

Загрузка списка баз для анализа в переменную down

In [2]:
import os
import win32api
import pandas as pd
import numpy as np

In [ ]:
paths_1 = input('Поиск будет осуществлен от расположения программы.\nУкажите из какой папки загружать файл\n(например, Загрузка\январь и февраль 2022): ')
extension = input('Укажите какое расширение у файлов (например, xlsx либо xls):\n')
paths_2 = './' + paths_1

Data_dlay_proverki = input('Дата текущего месяца (формат, ГГГГ-ММ, 2024-12): ')
Data_dlay_analiza_baza = input('Дата следующего месяца (формат, ГГГГ-ММ, 2025-01): ')
Data_dlay_proverki = Data_dlay_proverki + '-01'
Data_dlay_analiza_baza = Data_dlay_analiza_baza + '-01'

In [4]:
if paths_1.find(':') != -1: #Проверяет есть ли в указанном адресе ":", если его нет, то функция find() примет значение -1.
    down = pd.DataFrame([os.listdir(paths_1)]).T #создание матрицы со списком по указанному пути,
                                                 #далее список преобразуется в столбец
    down = down[down.iloc[:,0].str.contains(extension)] #Фильтруется список на содержание
                                                        #конкретных символов, указанных в переменной extension
    paths_3 = paths_1
else:
    down = pd.DataFrame([os.listdir(paths_2)]).T
    down = down[down.iloc[:,0].str.contains(extension)]
    paths_3 = paths_2

if paths_3[-1:] != '/': #добавляет в конце символ "/" если путь к файлам указан без закрывающего символа "/"
    paths_3 = paths_3 + '/'

In [ ]:
#Data_dlay_proverki = '2022-03-01' #Вводиться выше, при работе программы
Data_dlay_proverki = datetime.strptime(Data_dlay_proverki, "%Y-%m-%d")
Minus_day = pd.DataFrame([Data_dlay_proverki - timedelta(days=28)])

#Data_dlay_analiza_baza = '2022-04-01' #Вводиться выше, при работе программы
Data_dlay_analiza_baza = pd.DataFrame([datetime.strptime(Data_dlay_analiza_baza, "%Y-%m-%d")])

Minus_day.head()

    Загрузка Баз:

In [6]:
i = 0
for x in down[0]:
    if i == 0:
        baza = pd.read_excel(paths_3 + str(x))
    if i > 0:
        baza2 = pd.read_excel(paths_3 + str(x))
        baza = pd.concat([baza, baza2])
    i += 1

C:\Users\Mikhail\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


    Предварительный просмотр базы - первых 5 строк:

In [ ]:
baza.head()

In [8]:
baza.shape

(22309067, 27)

In [9]:
baza = baza.astype({'Номер заказа': str})

In [10]:
baza2 = baza[pd.to_datetime(baza['Дата']) < Data_dlay_analiza_baza.iloc[0, 0]]

In [11]:
baza2.shape

(11822712, 27)

In [ ]:
baza2.head()

                                                        1_1

In [13]:
file_1_1 = baza[(baza['Номер заказа'].str.contains('_') == False) & baza['Состояние'].str.contains('Отклонён')]
file_1_1 = file_1_1[pd.to_datetime(file_1_1['Дата']) < Data_dlay_analiza_baza.iloc[0, 0]]
file_1_1.index = pd.Series(np.arange(0,(file_1_1.shape)[0],1)) #Делаем индексы от нуля до конца новой базы.
file_1_1 = file_1_1.astype({'Код ответа процессинга': str}) #Далее делаем строки текстовыми,
file_1_1 = file_1_1.astype({'Процессинг': str})             #чтобы не округлялись.
file_1_1 = file_1_1.astype({'fare_id': str})

                                                        1_2

In [14]:
file_1_2 = baza[baza['Номер заказа'].str.contains('_') & baza['Состояние'].str.contains('Завершён')]
file_1_2.index = pd.Series(np.arange(0,(file_1_2.shape)[0],1)) #Делаем индексы от нуля до конца новой базы.
file_1_2 = file_1_2.astype({'Код ответа процессинга': str}) #Далее делаем строки текстовыми,
file_1_2 = file_1_2.astype({'Процессинг': str})             #чтобы не округлялись.
file_1_2 = file_1_2.astype({'fare_id': str})

                                                        3_3

In [15]:
file_3_3 = baza[baza['Номер заказа'].str.contains('manual') & baza['Состояние'].str.contains('Завершён')]
file_3_3.index = pd.Series(np.arange(0,(file_3_3.shape)[0],1)) #Делаем индексы от нуля до конца новой базы.
file_3_3 = file_3_3.astype({'Код ответа процессинга': str}) #Далее делаем строки текстовыми,
file_3_3 = file_3_3.astype({'Процессинг': str})             #чтобы не округлялись.
file_3_3 = file_3_3.astype({'fare_id': str})

$$manual$$

In [16]:
file_3_3_data = file_3_3
file_3_3_data['terminal_date2'] = pd.to_datetime(file_3_3['terminal_date'].astype(str).str[:10], format='%Y.%m.%d').copy()
file_3_3_data_min_or_ravno_28day = file_3_3_data[file_3_3_data['terminal_date2'] <= Minus_day.iloc[0,0]].copy()
file_3_3_data_max_28day = file_3_3_data[file_3_3_data['terminal_date2'] > Minus_day.iloc[0,0]].copy()

                                            Свод по общему файлу и manual:

In [ ]:
svod = pd.DataFrame({'Общий файл': [str(baza2['Сумма'].count()), str(baza2['Сумма'].sum())],
                     'manual': [file_3_3_data_min_or_ravno_28day['Сумма'].count(), file_3_3_data_min_or_ravno_28day['Сумма'].sum()]},
                     index = ['Количество','Сумма'])
svod.to_excel('./svod.xlsx', sheet_name='svod') #выгружаем результат
print('Дата за минусом 28 дней: ', Minus_day.iloc[0, 0], end = '\n\n')
print(svod)

$$Недоавторизованные$$

In [18]:
proverka = file_1_2
proverka['Номер заказа2'] = proverka['Номер заказа']
proverka2 = proverka.drop(proverka.iloc[:, 1:23], axis = 1)
proverka3 = proverka2
file_1_1 = file_1_1.astype({'Номер заказа': str})             #чтобы не округлялись.
proverka2 = proverka2.astype({'Номер заказа': str})
i = 0
for x2 in proverka2['Номер заказа']:
    proverka3.iloc[i,0] = (pd.DataFrame([x2[0: x2.find('_')]])).iloc[0,0]
    i += 1
file_1_3_v1 = pd.merge(file_1_1, proverka3, on='Номер заказа', how='left')
file_1_3_v1 = file_1_3_v1.fillna('пусто')
file_1_3_vND = file_1_3_v1.loc[file_1_3_v1['Номер заказа2'] == 'пусто']
file_1_3_vSTOP = file_1_3_v1.loc[file_1_3_v1['Номер заказа2'] != 'пусто']

In [ ]:
file_1_3_vND.head()

                                                           Стоп лист

In [24]:
file_3_1_vND = file_1_3_vND.copy()
file_3_1_vND['Дата2'] = pd.to_datetime(file_3_1_vND['Дата'].astype(str).str[:10], format='%Y.%m.%d')
file_3_1_vND['terminal_date2'] = pd.to_datetime(file_3_1_vND['terminal_date_x'].astype(str).str[:10], format='%Y-%m-%d')

file_3_1_vND_svod_min = file_3_1_vND.groupby(['pan_hash'])['terminal_date2'].min().copy()
file_3_1_vND_VPR_min = pd.merge(file_3_1_vND, file_3_1_vND_svod_min, on = 'pan_hash', how = 'left').copy()

file_3_1_vND = file_3_1_vND_VPR_min.copy()
file_3_1_vND['нам нужны 1 и более'] = file_3_1_vND['terminal_date2_x'] - file_3_1_vND['terminal_date2_y']
file_3_1_vND_STOPlist = file_3_1_vND[file_3_1_vND['нам нужны 1 и более'] > '0 day'].copy()

$$После24ч$$

In [25]:
file_3_2 = file_1_3_vND.copy()
file_3_2['Дата2'] = pd.to_datetime(file_3_2['Дата'].astype(str).str[:10], format='%Y.%m.%d')
file_3_2['terminal_date2'] = pd.to_datetime(file_3_2['terminal_date_x'].astype(str).str[:10], format='%Y-%m-%d')
file_3_2['нам нужны 2 и более'] = file_3_2['Дата2'] - file_3_2['terminal_date2']
file_3_2 = file_3_2[file_3_2['нам нужны 2 и более'] >= '2 day'].copy()

$$Код03$$

In [26]:
file_3_4 = file_1_3_vND[file_1_3_vND['Оригинальный код ответа процессинга'] == '03']
file_3_4_unikal = file_3_4.groupby(['Терминал'])['Терминал'].unique()

                                                        Выгрузка файлов:

In [27]:
file_1_1.to_excel('./1_1.xlsx', sheet_name='1_1', index=False) #1_1
file_1_2.to_excel('./1_2.xlsx', sheet_name='1_2', index=False) #1_2
file_3_3.to_excel('./3_3.xlsx', sheet_name='3_3', index=False) #3_3 - только фильтры по manual

file_1_3_vND.to_excel('./Недоавторизованные.xlsx', sheet_name='Недоавтор', index=False) #Недоавторизованные
file_3_1_vND_STOPlist.to_excel('./Стоп лист.xlsx', sheet_name='Стоп лист', index=False) #Стоп лист
file_3_2.to_excel('./После24ч.xlsx', sheet_name='После24ч', index=False) #После24ч
file_3_3_data_min_or_ravno_28day.to_excel('./manual.xlsx', sheet_name='manual', index=False) #manual
file_3_4.to_excel('./Код03.xlsx', sheet_name='Код03', index=False) #Код03

                                                    Свод2 по всем значениям:

In [ ]:
svod2 = pd.DataFrame({'Общий файл': [str(baza2['Сумма'].count()), str(baza2['Сумма'].sum())],
                     'Недоавторизованные': [file_1_3_vND['Сумма'].count(), file_1_3_vND['Сумма'].sum()],
                     'Стоп лист': [file_3_1_vND_STOPlist['Сумма'].count(), file_3_1_vND_STOPlist['Сумма'].sum()],
                     'После24ч': [file_3_2['Сумма'].count(), file_3_2['Сумма'].sum()],
                     'manual': [file_3_3_data_min_or_ravno_28day['Сумма'].count(), file_3_3_data_min_or_ravno_28day['Сумма'].sum()],
                     'Код03': [file_3_4['Сумма'].count(), file_3_4['Сумма'].sum()],},
                     index = ['Количество','Сумма'])
svod2.to_excel('./svod2.xlsx', sheet_name='svod2') #выгружаем результат
file_3_4_unikal.to_excel('./unikal_terminal.xlsx', sheet_name='unikal_t', index=False) #выгружаем уникальные терминалы
print('Дата за минусом 28 дней: ', Minus_day.iloc[0, 0], end = '\n')
print('Дата для анализа: ', Data_dlay_analiza_baza.iloc[0, 0], end = '\n\n')
print(svod2)
print('\nУникальные терминалы: ', file_3_4_unikal.index)